In [1]:
import requests, re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url1 = "https://www.fl.ru/projects/5218684/"
url3 = "https://www.fl.ru/projects/5214237/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36"
}

# Замаскированный запрос
response1 = requests.get(url1, headers=headers)

soup = BeautifulSoup(response1.text, "html.parser")

competition_name = soup.find_all("span")[7].get_text().strip()
customer = soup.find_all("a")[70].get_text().strip()
registration_date = soup.find("div", class_="text-gray-opacity-4 d-inline-block text-5 cursor-default").get_text().strip()[1:11]
budget_raw = soup.find_all("span")[9].get_text().strip()
budget = ""
for i in range(len(budget_raw)):
    if "0" <= budget_raw[i] <= "9":
        budget += budget_raw[i]
budget = int(budget)
description = soup.find("div", class_="text-5 text-dark").get_text().strip()
customer_url = soup.find_all("a")[70].get("href")
chapter = ""
for elem in soup.find_all("a", class_="text-decoration-none link-hover-danger text-blue cursor-pointer"):
    chapter += elem.get_text() + "/"
chapter = chapter[:-1]
participants = int(soup.find("span", attrs={"id": "stat-freelancers"}).get_text())
finish_date = soup.find("div", attrs={"class": "contest-period-in"}).find_all("p")[0].get_text()[-10:]
comments = int(soup.find("span", attrs={"id": "co-all"}).get_text())
comments_text = soup.find_all("ul", attrs={"class": "coments-list"})[0].get_text()
banned = int(soup.find("span", attrs={"id": "stat-banned"}).get_text())


df = pd.read_excel("Данные о конкурсах.xlsx")
del df["Unnamed: 0"]

df.loc[0, "URL"] = url1
df.loc[0, "Название конкурса"] = competition_name
df.loc[0, "Заказчик конкурса"] = customer
df.loc[0, "Дата регистрации конкурса"] = registration_date
df.loc[0, "Бюджет"] = budget
df.loc[0, "Описание"] = description
df.loc[0, "URL заказчика"] = "https://www.fl.ru" + customer_url
df.loc[0, "Разделы"] = chapter
df.loc[0, "Участники"] = participants
df.loc[0, "В бане"] = banned
df.loc[0, "Дата завершения приема заявок"] = finish_date
df.loc[0, "Количество комментариев"] = comments
df.loc[0, "Комментарии"] = comments_text



response2 = requests.get(df.loc[0, "URL заказчика"], headers=headers)

df.loc[0, "Платный аккаунт"] = "Платный аккаунт" in response2.text
df.loc[0, "Безопасные сделки"] = "Безопасные сделки" in response2.text
df.loc[0, "Телефон подтвержден"] = "Телефон подтвержден" in response2.text
df.loc[0, "Email подтвержден"] = "Email подтвержден" in response2.text
df.loc[0, "Паспортные данные проверены"] = "Паспортные данные проверены" in response2.text
df.loc[0, "Данные юридического лица/ИП проверены"] = "Данные юридического лица/ИП проверены" in response2.text

for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
    if "На сайте " in block.get_text():
        break
block = block.get_text()

if re.findall(r'На сайте .+\t', block)[0].strip() == "На сайте меньше месяца":
    df.loc[0, "Лет на сайте"] = 0
    df.loc[0, "Месяцев на сайте"] = 0

if re.findall(r'[\d]+ [лг]+', block) != []:
    df.loc[0, "Лет на сайте"] = int(re.findall(r'[\d]+ [лг]+', block)[0].strip().split(' ')[0])
else:
    df.loc[0, "Лет на сайте"] = 0
    
if re.findall(r'[\d]+ [м]+', block) != []:
    df.loc[0, "Месяцев на сайте"] = int(re.findall(r'[\d]+ [м]+', block)[0].strip().split(' ')[0])
else:
    df.loc[0, "Месяцев на сайте"] = 0
    

for block in BeautifulSoup(response2.text).find_all('td', class_="b-layout__td"):
    if "Рейтинг" in block.get_text() and "Отзывы" in block.get_text():
        break
block = re.sub(r'\s', r' ', block.get_text()).strip()
block = re.sub(r'[  ]+', r' ', block)
df.loc[0, "Рейтинг"] = float(re.findall(r'Рейтинг *\d*\.*\d+', block)[0].split(' ')[-1]) if 'Рейтинг' in block else 0
df.loc[0, "Безопасные сделки"] = int(re.findall(r'Безопасные сделки \d+', block)[0].split(' ')[-1]) if 'Безопасные сделки' in block else 0
df.loc[0, "Выбран[а] исполнителем"] = int(re.findall(r'Выбран[а] исполнителем \d+', block)[0].split(' ')[-1]) if 'Выбран' in block else 0
df.loc[0, "Отзывы+"] = int(re.findall(r'\+ \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
df.loc[0, "Отзывы-"] = int(re.findall(r'- \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0

df

,URL,Название конкурса,Заказчик конкурса,Дата регистрации конкурса,Бюджет,Описание,URL заказчика,Разделы,Участники,В бане,...,Телефон подтвержден,Email подтвержден,Паспортные данные проверены,Данные юридического лица/ИП проверены,Лет на сайте,Месяцев на сайте,Рейтинг,Выбран[а] исполнителем,Отзывы+,Отзывы-
0,https://www.fl.ru/projects/5218684/,Требуется разработать логотип «BIOL-technology...,Андрей Дронов,05.09.2023,30000,Требуется разработать логотип «BIOL-technology...,https://www.fl.ru/users/dronvrn333,Дизайн и Арт/Логотипы,41,0,...,True,True,False,False,2,7,573.6,0,2,0
1,https://www.fl.ru/projects/5214237/,Новый логотип для Chelay (расчески и товары дл...,Заказчик,26.08.2023,10000,Добрый день! Мы в поиске дизайна нашей мечты....,https://www.fl.ru/users/development19,Дизайн и Арт/Логотипы,33,0,...,True,True,False,False,0,3,422.3,0,0,0


In [3]:
response3 = requests.get(url3, headers=headers)

soup = BeautifulSoup(response3.text, "html.parser")
competition_name = soup.find_all("span")[7].get_text().strip()
customer = soup.find_all("a")[70].get_text().strip()
registration_date = soup.find("div", class_="text-gray-opacity-4 d-inline-block text-5 cursor-default").get_text().strip()[1:11]
budget_raw = soup.find_all("span")[9].get_text().strip()
budget = ""
for i in range(len(budget_raw)):
    if "0" <= budget_raw[i] <= "9":
        budget += budget_raw[i]
budget = int(budget)
description = soup.find("div", class_="text-5 text-dark").get_text().strip()
customer_url = soup.find_all("a")[70].get("href")

chapter = ""
for elem in soup.find_all("a", class_="text-decoration-none link-hover-danger text-blue cursor-pointer"):
    chapter += elem.get_text() + "/"
chapter = chapter[:-1]
participants = int(soup.find("span", attrs={"id": "stat-freelancers"}).get_text())
finish_date = soup.find("div", attrs={"class": "contest-period-in"}).find_all("p")[0].get_text()[-10:]
comments = int(soup.find("span", attrs={"id": "co-all"}).get_text())
comments_text = soup.find_all("ul", attrs={"class": "coments-list"})[0].get_text()
banned = int(soup.find("span", attrs={"id": "stat-banned"}).get_text())

df.loc[1, "URL"] = url3
df.loc[1, "Название конкурса"] = competition_name
df.loc[1, "Заказчик конкурса"] = customer
df.loc[1, "Дата регистрации конкурса"] = registration_date
df.loc[1, "Бюджет"] = budget
df.loc[1, "Описание"] = description
df.loc[1, "URL заказчика"] = "https://www.fl.ru" + customer_url
df.loc[1, "Разделы"] = chapter
df.loc[1, "Участники"] = participants
df.loc[1, "В бане"] = banned
df.loc[1, "Дата завершения приема заявок"] = finish_date
df.loc[1, "Количество комментариев"] = comments
df.loc[1, "Комментарии"] = comments_text



response4 = requests.get(df.loc[1, "URL заказчика"], headers=headers)

df.loc[1, "Платный аккаунт"] = "Платный аккаунт" in response4.text
df.loc[1, "Безопасные сделки"] = "Безопасные сделки" in response4.text
df.loc[1, "Телефон подтвержден"] = "Телефон подтвержден" in response4.text
df.loc[1, "Email подтвержден"] = "Email подтвержден" in response4.text
df.loc[1, "Паспортные данные проверены"] = "Паспортные данные проверены" in response4.text
df.loc[1, "Данные юридического лица/ИП проверены"] = "Данные юридического лица/ИП проверены" in response4.text

for block in BeautifulSoup(response4.text).find_all('td', class_="b-layout__td"):
    if "На сайте " in block.get_text():
        break
block = block.get_text()

if re.findall(r'На сайте .+\t', block)[0].strip() == "На сайте меньше месяца":
    df.loc[1, "Лет на сайте"] = 0
    df.loc[1, "Месяцев на сайте"] = 0

if re.findall(r'[\d]+ [лг]+', block) != []:
    df.loc[1, "Лет на сайте"] = int(re.findall(r'[\d]+ [лг]+', block)[0].strip().split(' ')[0])
else:
    df.loc[1, "Лет на сайте"] = 0
    
if re.findall(r'[\d]+ [м]+', block) != []:
    df.loc[1, "Месяцев на сайте"] = int(re.findall(r'[\d]+ [м]+', block)[0].strip().split(' ')[0])
else:
    df.loc[1, "Месяцев на сайте"] = 0
    

for block in BeautifulSoup(response4.text).find_all('td', class_="b-layout__td"):
    if "Рейтинг" in block.get_text() and "Отзывы" in block.get_text():
        break
block = re.sub(r'\s', r' ', block.get_text()).strip()
block = re.sub(r'[  ]+', r' ', block)
df.loc[1, "Рейтинг"] = float(re.findall(r'Рейтинг *\d*\.*\d+', block)[0].split(' ')[-1]) if 'Рейтинг' in block else 0
df.loc[1, "Безопасные сделки"] = int(re.findall(r'Безопасные сделки \d+', block)[0].split(' ')[-1]) if 'Безопасные сделки' in block else 0
df.loc[1, "Выбран[а] исполнителем"] = int(re.findall(r'Выбран[а] исполнителем \d+', block)[0].split(' ')[-1]) if 'Выбран' in block else 0
df.loc[1, "Отзывы+"] = int(re.findall(r'\+ \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0
df.loc[1, "Отзывы-"] = int(re.findall(r'- \d+', block)[0].split(' ')[-1]) if 'Отзывы' in block else 0

df

,URL,Название конкурса,Заказчик конкурса,Дата регистрации конкурса,Бюджет,Описание,URL заказчика,Разделы,Участники,В бане,...,Телефон подтвержден,Email подтвержден,Паспортные данные проверены,Данные юридического лица/ИП проверены,Лет на сайте,Месяцев на сайте,Рейтинг,Выбран[а] исполнителем,Отзывы+,Отзывы-
0,https://www.fl.ru/projects/5218684/,Требуется разработать логотип «BIOL-technology...,Андрей Дронов,05.09.2023,30000,Требуется разработать логотип «BIOL-technology...,https://www.fl.ru/users/dronvrn333,Дизайн и Арт/Логотипы,41,0,...,True,True,False,False,2,7,573.6,0,2,0
1,https://www.fl.ru/projects/5214237/,Новый логотип для Chelay (расчески и товары дл...,Заказчик,26.08.2023,10000,Добрый день! Мы в поиске дизайна нашей мечты....,https://www.fl.ru/users/development19,Дизайн и Арт/Логотипы,33,0,...,True,True,False,False,0,3,422.3,0,0,0


In [4]:
df.to_excel("Данные о конкурсах.xlsx")